In [57]:
# Import libraries
import psycopg2
import requests
from requests_html import HTMLSession
import sys
import os

# Set custom path for secrets
sys.path.insert(0,"/home/nuclear/Github/PythonPrograms/Secrets")

In [7]:
# Connect to database
import crawler_pg_secrets as p

pg_params = {
    'host' : p.host,
    'user' : p.user,
    'dbname' : p.dbname,
    'password' : p.password,
    'port' : p.port
}

conn =  psycopg2.connect(**pg_params)
conn.commit()
cur = conn.cursor()

In [49]:
table_name = 'indeed_horizon'

# Drop previous horizon table
sql_cmnd = "drop table " + table_name + ";"
try:
    cur.execute(sql_cmnd)
    print('success: ' + sql_cmnd)
except:
    print('FAIL: ' + sql_cmnd)
conn.commit()

# Create new horizon table
sql_cmnd = "create table "\
    + table_name\
    + " (id serial, url varchar(256), visited integer);"
try:
    cur.execute(sql_cmnd)
    print('success: ' + sql_cmnd)
except:
    print('FAIL: ' + sql_cmnd)
conn.commit()

# Add starting url to table
url='https://ca.indeed.com'



success: drop table indeed_horizon;
success: create table indeed_horizon (id serial, url varchar(256), visited integer);


In [64]:
# Set up requests with headers to bypass restrictions
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0'}
print(headers['User-Agent'])
# r=requests.get(url, headers=headers)

url='https://stomarket.com/market'
r=requests.get(url,headers=headers)


session = HTMLSession()
r = session.get(url)


session = requests.Session()
r = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})

print(r.headers)
print(r.status_code)

Mozilla/5.0 (X11; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0
{'Date': 'Fri, 11 Nov 2022 07:20:29 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'close', 'CF-Chl-Bypass': '1', 'Referrer-Policy': 'same-origin', 'Permissions-Policy': 'accelerometer=(),autoplay=(),camera=(),clipboard-read=(),clipboard-write=(),fullscreen=(),geolocation=(),gyroscope=(),hid=(),interest-cohort=(),magnetometer=(),microphone=(),payment=(),publickey-credentials-get=(),screen-wake-lock=(),serial=(),sync-xhr=(),usb=()', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'Server-Timing': 'cf-q-config;dur=5.9999983932357e-06', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v3?s=5%2BTQJK6TZpQRR%2FVHpXpa%2BZqVwyDopPsjimM6j8HgKPo5yUrgjwumA4BrI%2FQ8Z0nEwaSj7%2B1t56r2ll8rhuyxRUO6W519BQmRwkL5i5vCUe%2BvnXd

In [65]:
# Sample code from https://github.com/asifexplore/Indeed-Web-Crawler/blob/master/indeed_crawler_upgrade.py

import requests
from bs4 import BeautifulSoup
import pandas as pd

idArray = []

# Extracts everything from the page. Entire HTML 
def extractLayer1(page):
    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0'}
    url = f'https://www.indeed.com/jobs?q=Developer+USA&explvl=entry_level&start={page}'
    r = requests.get(url,headers)
    print(r.status_code)
    # Used to return status 200 if we are able to access the site properly. 
    # return r.status_code
    soup = BeautifulSoup(r.content,'html.parser')
    return soup

# Filters out the required "divs" from HTML (Passed from extract function)
def scrapLayer1(soup):
    # Finds all the div that has the respective class from the html taken from extract function. 
    divs = soup.find_all('a',class_ = 'tapItem')
    for item in divs: 
        if item.get('id') is not None:
            id = item.get('id')
            id = id.replace('job_','')
            idArray.append(id)
            link = f"https://www.indeed.com/jobs?q=Developer+USA&explvl=entry_level&vjk={id}"
        else:
            idArray.append("NTH")
        
        title = location = item.find('h2', class_ = 'jobTitle').text.strip()
        company = item.find('span', class_ = 'companyName').text.strip()
        date = item.find('span', class_ = 'date').text.strip()
        summary = item.find('div', class_ = 'job-snippet').text.strip().replace('\n','')
        
        location = item.find('div', class_ = 'companyLocation')
        if location is not None:
            location = location.text.strip()
        else:
            location = ''
        
        # Storing all data obtained into a Dictionary. 
        job = {
            'Job Title':title, 
            'Job Location':location, 
            'Job Company Name':company,
            'Job Description':summary, 
            'Job Date' : date, 
            # 'Job Salary':salary, 
            'Job Link':link,
        }

        joblist.append(job)
    return 

# Extracts everything from the page. Entire HTML 
def extract(id):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    url = f'https://www.indeed.com/viewjob?jk={id}'
    r = requests.get(url,headers)
    # Used to return status 200 if we are able to access the site properly. 
    # return r.status_code
    soup = BeautifulSoup(r.content,'html.parser')
    return soup

joblist = []

# Get all HTML details of job search 
# Create loop here to obtain more job info, to cover pagination. 
c = extractLayer1(0)
# Getting ID of all available jobs extracted
scrapLayer1(c)

for i in range(0,len(idArray)):
    d = extract(idArray[i])
    desc = d.find('div',class_ = 'jobsearch-jobDescriptionText').text
    joblist[i]['Job Description'] = desc
    print(joblist[i]["Job Description"])

df = pd.DataFrame(joblist)
print(df.head())
df.to_csv(r'C:\Users\Asif\Desktop\Indeed Crawler\jobs.csv')
print(joblist)

403
Empty DataFrame
Columns: []
Index: []
[]


In [67]:


input_skill = input('Enter your skill: ').strip()
place = input('Enter preferred location: ').strip()
no_of_pages = int(input('Enter # number of pages to scrape: '))

indeed_post = []

for page in range(no_of_pages):
    url = 'https://www.indeed.com/jobs?q=data%20analyst&l=US&fromage=1&vjk=91152078d1e41826' + input_skill + \
    '&1=' + place +'&sort=date' + '&start=' + str(page*10)
    
#request indeed url    
    html_content = requests.get(url)
    print(html_content.status_code)

Enter your skill: analyst
Enter preferred location: ontario
Enter # number of pages to scrape: 1
403
